In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
customers = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv')
transaction = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
articles = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv')

In [ ]:
customers = customers[['customer_id', 'club_member_status',
       'fashion_news_frequency', 'age']]

In [ ]:
customers.info()

In [ ]:
customers.loc[customers['fashion_news_frequency']=='NONE','fashion_news_frequency'] = float(0.0)
customers.loc[customers['fashion_news_frequency']=='Regularly','fashion_news_frequency'] = float(0.5)
customers.loc[customers['fashion_news_frequency']=='Monthly','fashion_news_frequency'] = float(1.0)

customers.loc[customers['club_member_status']=='ACTIVE','club_member_status'] = float(1.0)
customers.loc[customers['club_member_status']=='PRE-CREATE','club_member_status'] = float(0.5)
customers.loc[customers['club_member_status']=='LEFT CLUB','club_member_status'] = float(.0)

In [ ]:
customers = customers.dropna()
customers.info()

In [ ]:
transaction.columns

In [ ]:
articles.columns

In [ ]:
articles = articles[['article_id','product_type_no',
       'graphical_appearance_no', 'perceived_colour_value_id','section_no', 'garment_group_no']]

In [ ]:
count = transaction[['article_id']].value_counts().reset_index()
count.columns = ['article_id', 'cnt']
count = count.sort_values(by = 'cnt', ascending = False).head(24)

In [ ]:
transaction = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [ ]:
transaction = transaction[transaction['article_id'].isin(count['article_id'])]


In [ ]:
transaction = transaction.drop(['t_dat'], axis = 1)

In [ ]:
transaction.duplicated().sum()

In [ ]:
transaction = transaction.drop_duplicates(keep='first')

In [ ]:
transaction.info()

In [ ]:
master = transaction.merge(articles, on='article_id').merge(customers, on='customer_id')
master.info()

In [ ]:
master_sam = master.sample(n=10000)

In [ ]:
master_sam['index'] = np.arange(0, len(master_sam))
master_sam.head(10)

In [ ]:
master_ohe = pd.get_dummies(master_sam['article_id'])
master_ohe['index'] = np.arange(0, len(master_ohe))
master_ohe.head(20)

In [ ]:
master_sam = master_sam.merge(master_ohe, on = 'index')

In [ ]:
master_sam['fashion_news_frequency'] = master_sam['fashion_news_frequency'].astype('float64')
master_sam['club_member_status'] = master_sam['club_member_status'].astype('float64')

In [ ]:
master_sam.info()

put in model

In [ ]:
# split X and y into training and testing sets

X = master_sam.drop(['customer_id', 'article_id'], axis = 1)
y = master_sam['article_id']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [ ]:
# import DecisionTreeClassifier

from sklearn.tree import DecisionTreeClassifier

In [ ]:
# instantiate the DecisionTreeClassifier model with criterion gini index

clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)


# fit the model
clf_gini.fit(X_train, y_train)

In [ ]:
y_pred_gini = clf_gini.predict(X_test)

In [ ]:
y_pred_train_gini = clf_gini.predict(X_train)

y_pred_train_gini

In [ ]:
from sklearn.metrics import accuracy_score

print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test, y_pred_gini)))

In [ ]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_gini)))

In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(clf_gini.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf_gini.score(X_test, y_test)))